## Model Demo

- run the following cell to download dependencies and model

In [1]:
#@title just run this (it takes about 5 minutes)
!pip install -q transformers
!pip install -q sentencepiece

from tqdm import tqdm
from IPython.display import clear_output
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

path = 'pedramyazdipoor/parsbert_question_answering_PQuAD'
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForQuestionAnswering.from_pretrained(path)


# model_name = 'SajjadAyoubi/xlm-roberta-large-fa-qa'
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# clear_output()

# https://github.com/sajjjadayobi/PersianQA/blob/main/src/utils.py
from tqdm import tqdm
import torch

class AnswerPredictor:
  def __init__(self, model, tokenizer, device='cuda', n_best=10, max_length=512, stride=256, no_answer=False):
      """Initializes PyTorch Question Answering Prediction
      It's best to leave use the default values.
      Args:
          model: Fine-tuned torch model
          tokenizer: Transformers tokenizer
          device (torch.device): Running device
          n_best (int): Number of best possible answers
          max_length (int): Tokenizer max length
          stride (int): Tokenizer stride
          no_answer (bool): If True, model can return "no answer"
      """
      self.model = model.eval()
      self.tokenizer = tokenizer
      self.device = device
      self.max_length = max_length
      self.stride = stride
      self.no_answer = no_answer
      self.n_best = n_best


  def model_pred(self, questions, contexts, batch_size=1):
      n = len(contexts)
      if n%batch_size!=0:
          raise Exception("batch_size must be divisible by sample length")

      tokens = self.tokenizer(questions, contexts, add_special_tokens=True,
                              return_token_type_ids=True, return_tensors="pt", padding=True,
                              return_offsets_mapping=True, truncation="only_second",
                              max_length=self.max_length, stride=self.stride)

      start_logits, end_logits = [], []
      for i in tqdm(range(0, n-batch_size+1, batch_size)):
          with torch.no_grad():
              out = self.model(tokens['input_ids'][i:i+batch_size].to(self.device),
                          tokens['attention_mask'][i:i+batch_size].to(self.device),
                          tokens['token_type_ids'][i:i+batch_size].to(self.device))

              start_logits.append(out.start_logits)
              end_logits.append(out.end_logits)

      return tokens, torch.stack(start_logits).view(n, -1), torch.stack(end_logits).view(n, -1)


  def __call__(self, questions, contexts, batch_size=1, answer_max_len=100):
      """Creates model prediction

      Args:
          questions (list): Question strings
          contexts (list): Contexts strings
          batch_size (int): Batch size
          answer_max_len (int): Sets the longests possible length for any answer

      Returns:
          dict: The best prediction of the model
              (e.g {0: {"text": str, "score": int}})
      """
      tokens, starts, ends = self.model_pred(questions, contexts, batch_size=batch_size)
      start_indexes = starts.argsort(dim=-1, descending=True)[:, :self.n_best]
      end_indexes = ends.argsort(dim=-1, descending=True)[:, :self.n_best]

      preds = {}
      for i, (c, q) in enumerate(zip(contexts, questions)):
          min_null_score = starts[i][0] + ends[i][0] # 0 is CLS Token
          start_context = tokens['input_ids'][i].tolist().index(self.tokenizer.sep_token_id)

          offset = tokens['offset_mapping'][i]
          valid_answers = []
          for start_index in start_indexes[i]:
              # Don't consider answers that are in questions
              if start_index<start_context:
                  continue
              for end_index in end_indexes[i]:
                  # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                  # to part of the input_ids that are not in the context.
                  if (start_index >= len(offset) or end_index >= len(offset)
                      or offset[start_index] is None or offset[end_index] is None):
                      continue
                  # Don't consider answers with a length that is either < 0 or > max_answer_length.
                  if end_index < start_index or (end_index-start_index+1) > answer_max_len:
                      continue

                  start_char = offset[start_index][0]
                  end_char = offset[end_index][1]
                  valid_answers.append({"score": (starts[i][start_index] + ends[i][end_index]).item(),
                                        "text": c[start_char: end_char]})

          if len(valid_answers) > 0:
              best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
          else:
              best_answer = {"text": "", "score": min_null_score}

          if self.no_answer:
              preds[i] = best_answer if best_answer["score"] >= min_null_score else {"text": "", "score": min_null_score}
          else:
              preds[i] = best_answer

      return preds

predictor = AnswerPredictor(model, tokenizer, device='cpu', n_best=10, no_answer=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/651M [00:00<?, ?B/s]

Some weights of the model checkpoint at pedramyazdipoor/parsbert_question_answering_PQuAD were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at pedramyazdipoor/parsbert_question_answering_PQuAD and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be 

In [ ]:
#@title Paste your Context and ask a Question
context = "باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق‌ترین تیم تاریخ فوتبال اسپانیا و موفق‌ترین تیم تاریخ فوتبال اروپا و موفق‌ترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن‌ها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه‌های جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی است. همچنین رئال مادرید برنده بهترین باشگاه قرن از سوی فیفا شده‌است. همچنین کلمهٔ رئال در زبان اسپانیایی، به معنی «سلطنتی» است، این لقب را شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد. همچنین در همان سال، شکل یک تاج نیز بر روی آرم این باشگاه قرار گرفت. رئال مادرید سال‌هاست که در لالیگا، از تیم‌های قدرتمند محسوب می‌شود و رقیب اصلی این تیم نیز، بارسلونا است. از نظر درآمد، رئال مادرید با ۵۵۷ میلیون یورو درآمد در فصل ۱۵–۲۰۱۴ در صدر پردرآمدترین باشگاه‌های فوتبال جهان بوده‌است." #@param {type:"string"}
question = "پایتخت اسپانیا کجاست؟" #@param {type:"string"}

preds = predictor([question], [context], batch_size=1)

print('\n\n Model Prediction: ', preds[0]['text'].strip())

100%|██████████| 1/1 [00:02<00:00,  2.67s/it]



 Model Prediction:  سوپرکاپ


## dataset

In [2]:
!pip install -q transformers
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", w

In [ ]:
!pip uninstall -y pyarrow

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0


In [ ]:
!pip install pyarrow --upgrade

  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (39.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

In [4]:
datasets = load_dataset("SajjadAyoubi/persian_qa")

Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

In [ ]:
datasets['validation'][8]

{'id': 9116,
 'title': 'باشگاه فوتبال رئال مادرید',
 'context': 'باشگاه فوتبال رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق\u200cترین تیم تاریخ فوتبال اسپانیا و موفق\u200cترین تیم تاریخ فوتبال اروپا و موفق\u200cترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن\u200cها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه\u200cهای جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی است. همچنین رئال مادرید برنده بهترین باشگاه قرن از سوی فیفا شده\u200cاست. همچنین کلمهٔ رئال در زبان اسپانیایی، به معنی «سلطنتی» است، این لقب را شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد. همچنین در همان سال، شکل یک تاج نیز بر روی آرم این باشگاه قرار گرفت. رئال مادرید سال\u200cهاست که در لالیگا، از تیم\u200cهای قدرتمند محسوب می\u200cشود و رقیب اصلی این تیم نیز، بارسل

In [ ]:
datasets['validation'][10]['answers']['text']

['بارسلونا', 'باشگاه فوتبال بارسلونا']

In [5]:
contexts = []
questions = []
answers = []
test_data = datasets['validation']
for p in test_data:
    if p['answers']['text']:
      contexts.append(p["context"])
      questions.append(p["question"])
      answers.append([p["answers"]["text"]][0])
answers[0]

['مادرید', 'مادرید']

In [ ]:
predictions = []
for question, context in zip(questions, contexts):
    preds = predictor([question], [context], batch_size=1)
    predictions.append(preds[0]['text'].strip())

In [ ]:
predictions[10]

'عضو بوده که تیم فوتبال بارسلونا بخشی از آن می\u200cباشد. تیم این باشگاه در لالیگا بازی می\u200cکند و جزو سه تیمی است که هرگز به دستهٔ پایین\u200cتر سقوط نکرده\u200cاست. شهرت تاریخی تیم این باشگاه به\u200cدلیل شیوه حمله\u200cای ماهرانه و جذابش بوده که تأکیدش را بر بازی باز قرار می\u200cدهد. این باشگاه با نام باشگاه فوتبال بارسلونا در سال ۱۸۹۹ توسط عده\u200cای سوئیسی، انگلیسی و اسپانیایی و به رهبری خوان گمپر تأسیس شد. بارسا به نوعی،'

In [ ]:
predictions[0] == answers[0][0]

False

In [ ]:
len(predictions) == len(answers)

True

In [ ]:
def exact_match(predictions, true_answers_list):
    matches = []
    for true_answers in true_answers_list:
        for answer in true_answers:
            if answer in predictions:
                matches.append(1)
                break
        else:
            matches.append(0)
    return sum(matches) / len(matches)


def f1_score(pred, true_answers):
    max_f1 = 0.0
    pred_tokens = pred.split()
    for true in true_answers:
        true_tokens = true.split()
        common_tokens = set(pred_tokens) & set(true_tokens)
        if not common_tokens:
            break
        precision = len(common_tokens) / len(pred_tokens)
        recall = len(common_tokens) / len(true_tokens)
        f1 = 2 * (precision * recall) / (precision + recall)
        if f1 > max_f1:
            max_f1 = f1
    return max_f1

em_score = exact_match(predictions, answers)
f1_scores = [f1_score(p, t) for p, t in zip(predictions, answers)]

average_f1_score = sum(f1_scores) / len(f1_scores)

print(f"Exact Match Score: {em_score}")
print(f"Average F1 Score: {average_f1_score}")

Exact Match Score: 0.0
Average F1 Score: 0.06520521670521198


## Trainer


In [74]:
model_checkpoint = "pedramyazdipoor/parsbert_question_answering_PQuAD"
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.
batch_size = 8
lr = 3e-5
epoch = 2

In [75]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [76]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [77]:
# the datasets library does cashing itself, batched is multitreading for fast-tokenizer
tokenized_ds = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

In [78]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at pedramyazdipoor/parsbert_question_answering_PQuAD were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at pedramyazdipoor/parsbert_question_answering_PQuAD and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be 

In [79]:
args = TrainingArguments(
    f"result",
    evaluation_strategy = "epoch",
    fp16=True,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.0001)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [80]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer)

In [81]:
# start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.035500,1.516052
2,1.181300,1.545200


TrainOutput(global_step=2252, training_loss=1.6998148997765874, metrics={'train_runtime': 714.7926, 'train_samples_per_second': 25.205, 'train_steps_per_second': 3.151, 'total_flos': 2568228663292608.0, 'train_loss': 1.6998148997765874, 'epoch': 2.0})

### After Fine Tuning

In [82]:
from transformers import pipeline

In [83]:
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [84]:
#@title Paste your Context and ask a Question
context = "باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق‌ترین تیم تاریخ فوتبال اسپانیا و موفق‌ترین تیم تاریخ فوتبال اروپا و موفق‌ترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن‌ها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه‌های جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی است. همچنین رئال مادرید برنده بهترین باشگاه قرن از سوی فیفا شده‌است. همچنین کلمهٔ رئال در زبان اسپانیایی، به معنی «سلطنتی» است، این لقب را شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد. همچنین در همان سال، شکل یک تاج نیز بر روی آرم این باشگاه قرار گرفت. رئال مادرید سال‌هاست که در لالیگا، از تیم‌های قدرتمند محسوب می‌شود و رقیب اصلی این تیم نیز، بارسلونا است. از نظر درآمد، رئال مادرید با ۵۵۷ میلیون یورو درآمد در فصل ۱۵–۲۰۱۴ در صدر پردرآمدترین باشگاه‌های فوتبال جهان بوده‌است." #@param {type:"string"}
question = "پایتخت اسپانیا کجاست؟" #@param {type:"string"}

result = qa_pipeline(question=question, context=context)

print(f"Answer: {result['answer']}")

Answer: مادرید


In [85]:
predictions = []
for question, context in zip(questions, contexts):
    preds = qa_pipeline(question=question, context=context)
    predictions.append(preds['answer'].strip())

In [86]:
predictions[10]

'نهادی برای ترویج فرهنگ کاتالان و کاتالانیسم است'

In [87]:
len(predictions) == len(answers)

True

In [88]:
def exact_match(predictions, true_answers_list):
    matches = []
    for true_answers in true_answers_list:
        for answer in true_answers:
            if answer in predictions:
                matches.append(1)
                break
        else:
            matches.append(0)
    return sum(matches) / len(matches)


def f1_score(pred, true_answers):
    max_f1 = 0.0
    pred_tokens = pred.split()
    for true in true_answers:
        true_tokens = true.split()
        common_tokens = set(pred_tokens) & set(true_tokens)
        if not common_tokens:
            break
        precision = len(common_tokens) / len(pred_tokens)
        recall = len(common_tokens) / len(true_tokens)
        f1 = 2 * (precision * recall) / (precision + recall)
        if f1 > max_f1:
            max_f1 = f1
    return max_f1

em_score = exact_match(predictions, answers)
f1_scores = [f1_score(p, t) for p, t in zip(predictions, answers)]

average_f1_score = sum(f1_scores) / len(f1_scores)

print(f"Exact Match Score: {em_score}")
print(f"Average F1 Score: {average_f1_score}")

Exact Match Score: 0.5299539170506913
Average F1 Score: 0.7553140101346242
